# Image Object Recognition Trainer

## Stage 1: Annotation

In [1]:
import os
import cv2
import numpy as np

In [2]:
training_data = "./daylight_images"
image_width = 1080
image_height = 1920

images = [f for f in os.listdir(training_data) if f[-4:] == ".jpg"]
len(images)

7020

In [3]:
def is_daylight(image, brightness_threshold=100, blue_threshold=30):
    # Convert to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Extract the Value (brightness) channel
    brightness = hsv[:, :, 2]

    # Calculate average brightness
    avg_brightness = np.mean(brightness)

    # Convert to RGB and calculate average blue intensity
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    blue_channel = rgb[:, :, 2]
    avg_blue_intensity = np.mean(blue_channel)

    # Determine daylight
    is_day = avg_brightness > brightness_threshold and avg_blue_intensity > blue_threshold

    return is_day, avg_brightness, avg_blue_intensity

In [4]:
import json
from dataclasses import dataclass
from io import BytesIO
import time

from fastapi import FastAPI, Request
from fastapi.responses import FileResponse, StreamingResponse, Response


app = FastAPI()
annotations = []
annotation_index = 0


@dataclass
class Annotation:
    coords: dict[str, tuple[int, int]]
    value: str

    def __post_init__(self):
        if self.value == "":
            self.value = "0"


@app.get('/htmx.min.js')
async def getHTMX():
    return FileResponse("webapp_templates/htmx.min.js", media_type="application/javascript")


@app.get('/bootstrap.min.js')
async def getBootstrapJS():
    return FileResponse("webapp_templates/bootstrap.min.js", media_type="application/javascript")


@app.get('/bootstrap.min.css')
async def getBootstrapCSS():
    return FileResponse("webapp_templates/bootstrap.min.css", media_type="text/css")


@app.get('/')
async def getImageAnnotator():
    read_annotations(images[annotation_index])
    return FileResponse("webapp_templates/image_annotator.html", media_type="text/html")


@app.get('/annotator')
async def getImageAnnotator():
    return FileResponse("webapp_templates/annotator.html", media_type="text/html", headers={"Expires": "0"})


def render_annotations():
    with open("webapp_templates/annotation.html") as f:
        template = f.read()
    return "\n".join([template.format(
        annotation_index=idx,
        annotation_coordinates=json.dumps(annotation.coords),
        annotation_value=annotation.value)
    for idx, annotation in enumerate(annotations)])


def write_annotations(image_path):
    image_annotation_path = f"{training_data}/{image_path[:-4]}.txt"
    image_annotations = []
    for annotation in annotations:
        p0 = annotation.coords['p0']
        p1 = annotation.coords['p1']
        p0_x, p0_y = p0
        p1_x, p1_y = p1
        width, height = [abs(float(d1) - float(d0)) for d0, d1 in zip([p0_x, p0_y], [p1_x, p1_y])]
        c_x = (float(p0_x) + width / 2) / image_width
        c_y = (float(p0_y) + height / 2) / image_height
        width /= image_width
        height /= image_height
        print(f"Writing notation from {annotation.coords}: {c_x} {c_y} {width} {height}")
        image_annotations.append(f"{annotation.value} {c_x} {c_y} {width} {height}\n")
    with open(image_annotation_path, "w") as f:
        f.write("\n".join(image_annotations))


def read_annotations(image_path):
    image_annotation_path = f"{training_data}/{image_path[:-4]}.txt"
    global annotations
    annotations = []
    
    if not os.path.exists(image_annotation_path):
        return

    with open(image_annotation_path) as f:
        stored_image_annotations = f.read()
    print(f"Reading {stored_image_annotations}")
    for anno_line in stored_image_annotations.split("\n"):
        try:
            value, c_x, c_y, width, height = anno_line.split(" ")
        except:
            continue
        c_x = float(c_x)
        c_y = float(c_y)
        width = float(width)
        height = float(height)
        c_x *= image_width
        c_y *= image_height
        width *= image_width
        height *= image_height
        p0 = [c_x - width / 2, c_y - height / 2]
        p1 = [c_x + width / 2, c_y + height / 2]
        annotations.append(Annotation(coords={"p0": p0, "p1": p1}, value=value))


@app.get("/annotation_index")
async def getAnnotationIndex():
    return f"{annotation_index} of {len(images) - 1}"


@app.post('/annotations')
async def setAnnotations(request: Request):
    # Path to your image file
    raw_body = await request.body()  # Get the raw bytes of the request body
    body_str = raw_body.decode("utf-8")  # Decode bytes to a string
    global annotations
    annotations = [Annotation(**a) for a in json.loads(body_str)]  # Parse the JSON data
    print(f"Updated annotations: {annotations}")
    write_annotations(images[annotation_index])
    return Response(render_annotations(), media_type="text/html")


@app.get('/annotations')
async def getAnnotations(request: Request):
    print(f"Current annotations: {annotations}")
    return Response(render_annotations(), media_type="text/html")


@app.get('/clearannotations')
async def clearAnnotations():
    global captureInProgress, annotations
    captureInProgress = False
    annotations = []
    return Response("Success!", media_type="text/html")

@app.get("/image_list")
async def get_image_list():
    rendered = ""
    for idx, image in enumerate(images):
        if idx == annotation_index:
            image = f"<b>{image}</b>"
        rendered = f'{rendered}\n<a href="/image/{idx}">{image}</a><br>'
    return Response(rendered, media_type="text/html")


def generate_image_stream():
    while True:
        image_name = images[annotation_index]
        image_path = f"training_data/{image_name}"
    
        image = cv2.imread(image_path)

        for annotation in annotations:
            points = annotation.coords
            # Convert points to integer tuples for OpenCV
            p0 = tuple(map(int, points["p0"]))
            p1 = tuple(map(int, points["p1"]))
            
            # Draw the rectangle
            color = (0, 255, 0)  # Green color in BGR
            thickness = 2        # Thickness of the rectangle border
            cv2.rectangle(image, p0, p1, color, thickness)

        # Encode the image as JPEG
        _, img_encoded = cv2.imencode('.jpg', image)

        # Yield the image bytes
        yield (b"--frame\r\nContent-Type: image/jpeg\r\n\r\n" + img_encoded.tobytes() + b"\r\n")

        # Wait before sending the next frame (simulate dynamic updates)
        time.sleep(0.2)

    
@app.get("/image")
async def get_image():
    return StreamingResponse(generate_image_stream(), media_type="multipart/x-mixed-replace; boundary=frame")


@app.get("/next")
async def next_image():
    global annotation_index
    annotation_index = min(len(images), annotation_index + 1)
    print(f"Updated annotation_index: {annotation_index}")
    read_annotations(images[annotation_index])
    return FileResponse("webapp_templates/annotator.html", media_type="text/html", headers={"Expires": "0"})
    
@app.get("/prev")
async def prev_image():
    global annotation_index
    annotation_index = max(0, annotation_index - 1)
    print(f"Updated annotation_index: {annotation_index}")
    read_annotations(images[annotation_index])
    return FileResponse("webapp_templates/annotator.html", media_type="text/html", headers={"Expires": "0"})
    
@app.get("/image/{idx}")
async def set_image(idx: int):
    global annotation_index
    annotation_index = min(len(images), max(0, idx))
    print(f"Updated annotation_index: {annotation_index}")
    read_annotations(images[annotation_index])
    return FileResponse("webapp_templates/image_annotator.html", media_type="text/html", headers={"Expires": "0"})

In [ ]:
if __name__ == "__main__":
    import uvicorn
    import nest_asyncio
    nest_asyncio.apply()
    uvicorn.run(app, host="127.0.0.1", port=5000)

INFO:     Started server process [357832]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5000 (Press CTRL+C to quit)


INFO:     127.0.0.1:51898 - "GET /?annotation_index=50 HTTP/1.1" 200 OK
INFO:     127.0.0.1:51898 - "GET /annotator HTTP/1.1" 200 OK
INFO:     127.0.0.1:51898 - "GET /image HTTP/1.1" 200 OK
INFO:     127.0.0.1:51914 - "GET /annotation_index HTTP/1.1" 200 OK
Current annotations: []
INFO:     127.0.0.1:51914 - "GET /annotations HTTP/1.1" 200 OK
INFO:     127.0.0.1:51914 - "GET /image_list HTTP/1.1" 200 OK
Updated annotation_index: 50
INFO:     127.0.0.1:51914 - "GET /image/50 HTTP/1.1" 200 OK
INFO:     127.0.0.1:51914 - "GET /annotator HTTP/1.1" 200 OK
INFO:     127.0.0.1:51914 - "GET /image HTTP/1.1" 200 OK
INFO:     127.0.0.1:51918 - "GET /annotation_index HTTP/1.1" 200 OK
Current annotations: []
INFO:     127.0.0.1:51920 - "GET /annotations HTTP/1.1" 200 OK
INFO:     127.0.0.1:51920 - "GET /image_list HTTP/1.1" 200 OK
Updated annotation_index: 79
INFO:     127.0.0.1:51920 - "GET /image/79 HTTP/1.1" 200 OK
INFO:     127.0.0.1:51920 - "GET /annotator HTTP/1.1" 200 OK
INFO:     127.0.0.1